### Lab1 : Building an AI Agent using Amazon Bedrock Agents

This notebook demonstrates the implementation of an AI Agent designed to guide potential home buyers through the mortgage pre-approval process. The assistant serves two primary functions:

1. New Loan Application: Helps users initiate a new home loan application by collecting relevant financial information and personal details
2. Loan Status Inquiry: Enables users to check the status of their existing home loan applications

The notebook showcases how AI agents can be configured to interact with external tools and APIs. While the agent architecture is fully implemented, the notebook uses simulated tool implementations rather than connecting to actual financial services.

Step 1 : Install the pre-requisites

In [ ]:
%pip install --upgrade -q -r ../src/requirements.txt

Step 2 : Restart the kernel. If this does not work, click Kernel -> Restart Kernel option at the top

In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True)

Step 3: Import libraries and helper functions

In [ ]:
import os
import time
import boto3
import logging
import botocore
import json
from textwrap import dedent

%load_ext autoreload
%autoreload 2

In the following cell, we add `bedrock_agent_helper.py` on Python path. This file contain helper classes focused on making labs experience smooth. 

All interactions with Bedrock will be handled by these classes.

Following methods will be used in this lab:

From `bedrock_agent.py`:
- `create_agent`: Create a new agent and respective IAM roles
- `add_action_group_with_lambda`: Create a lambda function and add it as an action group for a previous created agent
- `create_agent_alias`: Create an alias for this agent
- `invoke`: Execute agent

In [ ]:
import sys
sys.path.insert(0, '..') 

from src.utils.bedrock_agent import AgentsForAmazonBedrock

Step 4: Create Agent

Create the bedrock agent. Review the instructions for the model and the list of tools provided. Each tool corresponds to a method in the python file we create  above (which becomes the lambda)

In [ ]:
mortgage_application_agent_name = "mortgage_application_agent"

mortgage_application_lambda_name = "fn_mortgage_application"

mortgage_application_agent_role_name = f'AmazonBedrockExecutionRoleForAgents_{mortgage_application_agent_name}'

agent_foundation_model = ["anthropic.claude-3-5-haiku-20241022-v1:0"]

In [ ]:
agent_description = """Handle conversations about mortgage loan application assistant and applications for new mortgages."""

agent_instruction = """
Instructions: 
You are a mortgage bot for creating, managing, and completing an application for a new mortgage. you greet the customer and state your function before your answer. 
First, ask customers for their customer id. If they don't have any then you use the tool to create a new customer id and tell the user that you have created a new customer id and show it to them.
Next, you ask for their name, age, annual income and annual expense. Ask one question at a time. If they cant answer any of the questions then its fine, you just move forward. 
Once you have all the information use the tool to create a new loan application for this customer. 
After creating the loan application give the customer their newly created customer id if they didn't provide one initially.

Core behaviors:
1. Always use available information systems before asking customers for additional details
2. Maintain a professional yet conversational tone
3. Provide clear, direct answers without referencing internal systems or data sources
4. Present information in an easy-to-understand manner
5. Use code generation and interpretation capabilities for any on the fly calculation. DO NOT try to calculate things by yourself
6. Final response should not include your internal thought process


Response style:
- Be helpful and solution-oriented
- Use clear, non-technical language
- Focus on providing actionable insights
- Maintain natural conversation flow
- Be concise yet informative 
- do not add extra information not required by the user"""

In [ ]:
agents = AgentsForAmazonBedrock()

mortgage_application_agent = agents.create_agent(
    mortgage_application_agent_name,
    agent_description,
    agent_instruction,
    agent_foundation_model,
    code_interpretation=True
)

mortgage_application_agent_id = mortgage_application_agent[0]

print("Agent created with ID: ",mortgage_application_agent_id)

Step 5: Creating a Lambda function

In order to enable the agent to execute tasks, we will create an AWS Lambda function that implements the tasks execution. We will then provide this lambda function to the agent action group. You can find more information on how to use action groups to define actions that your agent can perform [here](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-create.html)

In [ ]:
%%writefile mortgage_application_function.py
import json
from datetime import datetime, timedelta
import random

NO_CUSTOMER_MESSAGE = "Invalid function call, since no customer ID was provided as a parameter, and it was not passed in session state."

def get_named_parameter(event, name):
    if 'parameters' in event:
        if event['parameters']:
            for item in event['parameters']:
                if item['name'] == name:
                    return item['value']
        return None
    else:
        return None
    
def populate_function_response(event, response_body):
    return {'response': {'actionGroup': event['actionGroup'], 'function': event['function'],
                'functionResponse': {'responseBody': {'TEXT': {'body': str(response_body)}}}}}

def get_mortgage_app_doc_status(customer_id):
    # TODO: Implement the actual logic to retrieve the document status for the given customer ID
    return [
        {
            "type": "proof_of_income",
            "status": "COMPLETED"
        },
        {
            "type": "employment_information",
            "status": "MISSING"
        },
        {
            "type": "proof_of_assets",
            "status": "COMPLETED"
        },
        {
            "type": "credit_information",
            "status": "COMPLETED"
        }
    ]


def get_application_details(customer_id):
    return {
        "customer_id": customer_id,
        "application_id": "998776",
        "application_date": datetime.today() - timedelta(days=35), # simulate app started 35 days ago
        "application_status": "IN_PROGRESS",
        "application_type": "NEW_MORTGAGE",
        "name" : "Mithil"
    }

def create_customer_id():
    return "123456"

def create_loan_application(customer_id, name, age, annual_income, annual_expense):
    print(f"creating loan application for customer: {customer_id}...")
    print(f"customer name: {name}")
    print(f"customer age: {age}")
    print(f"customer annual income: {annual_income}")
    print(f"customer annual expense: {annual_expense}")

    
def lambda_handler(event, context):
    print(event)
    function = event['function']

    if function == 'get_mortgage_app_doc_status':
        customer_id = get_named_parameter(event, 'customer_id')
        if not customer_id:
            # pull customer_id from session state variables if it was not supplied
            session_state = event['sessionAttributes']
            if session_state is None:
                return NO_CUSTOMER_MESSAGE
            else:
                if 'customer_id' in session_state:
                    customer_id = session_state['customer_id']
                else:
                    # return NO_CUSTOMER_MESSAGE
                    # for now, graceully just default, since this is just a toy example
                    customer_id = "123456"
            print(f"customer_id was pulled from session state variable = {customer_id}")
        result = get_mortgage_app_doc_status(customer_id)

    elif function == 'get_application_details':
        customer_id = get_named_parameter(event, 'customer_id')
        if not customer_id:
            # pull customer_id from session state variables if it was not supplied
            session_state = event['sessionAttributes']
            if session_state is None:
                return NO_CUSTOMER_MESSAGE
            else:
                if 'customer_id' in session_state:
                    customer_id = session_state['customer_id']
                else:
                    # return NO_CUSTOMER_MESSAGE
                    # for now, graceully just default, since this is just a toy example
                    customer_id = "123456"
            print(f"customer_id was pulled from session state variable = {customer_id}")
        result = get_application_details(customer_id)
    elif function == 'create_customer_id':
        result = create_customer_id()

    elif function == 'create_loan_application':
        customer_id = get_named_parameter(event, 'customer_id')
        if not customer_id:
            # pull customer_id from session state variables if it was not supplied
            session_state = event['sessionAttributes']
            if session_state is None:
                return NO_CUSTOMER_MESSAGE
            else:
                if 'customer_id' in session_state:
                    customer_id = session_state['customer_id']
                else:
                    # return NO_CUSTOMER_MESSAGE
                    # for now, graceully just default, since this is just a toy example
                    customer_id = "XXXXXX"
            print(f"customer_id was pulled from session state variable = {customer_id}")
        name = get_named_parameter(event, 'name')
        age = get_named_parameter(event, 'age')
        annual_income = get_named_parameter(event, 'annual_income')
        annual_expense = get_named_parameter(event, 'annual_expense')
       
        result = create_loan_application(customer_id, name, age, annual_income, annual_expense)
    else:
        raise Exception(f"Unrecognized function: {function}")


    response = populate_function_response(event, result)
    print(response)
    return response


Step 6 : Defining available actions

Next we will define the available actions that an agent can perform using [Function Details](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-action-function.html). You can also do this task using OpenAPI Schemas, which can be very useful if you already have an OpenAPI schema available for your application.

When creating your function details, it is important to provide clear descriptions for the function and for its parameters, as your agent depends on them to correctly orchestrate the tasks to be executed

In [ ]:
functions_def=[
        {
            "name": "get_mortgage_app_doc_status",
            "description": """
Retrieves the list of required documents for a mortgage application in process, 
along with their respective statuses (COMPLETED or MISSING). 
The function takes a customer ID, but it is purely optional. The funciton
implementation can retrieve it from session state instead.
This function returns a list of objects, where each object represents 
a required document type. 
The required document types for a mortgage application are: proof of income, employment information, 
proof of assets, and credit information. Each object in the returned list contains the type of the 
required document and its corresponding status. """,
            "parameters": {
                "customer_id": {
                    "description": """
        The unique identifier of the customer whose mortgage application document status is to be retrieved.""",
                    "type": "string",
                    "required": False
                }
            }
        },
        {
            "name": "get_application_details",
            "description": """
Retrieves the details about an application for a new mortgage.
The function takes a customer ID, but it is purely optional. The funciton
implementation can retrieve it from session state instead. Details include
the application ID, application date, application status, application type,
application amount, application tentative rate, and application term in years. """,
            "parameters": {
                "customer_id": {
                    "description": """
        The unique identifier of the customer whose mortgage application details is to be retrieved.""",
                    "type": "string",
                    "required": False
                }
            }
        },
        {
            "name": "create_loan_application",
            "description": """
            Creates a new loan application using the details provided. The details include the name,
            age, customer_id, annual_income and annual_expense
            """,
            "parameters": {
                "name": {
                    "description": """
        The name of the customer.""",
                    "type": "string",
                    "required": True
                },
                "age": {
                    "description": """
        The age of the customer.""",
                    "type": "integer",
                    "required": True
                },
                "customer_id": {
                    "description": """
        The unique identifier of the customer.""",
                    "type": "string",
                    "required": True
                },
                "annual_income": {
                    "description": """
        The annual income of the customer.""",
                    "type": "integer",
                    "required": True
                },
                "annual_expense": {
                    "description": """
        The annual expense of the customer.""",
                    "type": "integer",
                    "required": True
                }
            }
        },
        {
          "name": "create_customer_id",
          "description": "Use this function to create a new customer id",
          "parameters": {},
          "requireConfirmation": "DISABLED"
        }
    ]

Step 7: Add the Lambda function and the function details as an action group for this agent and prepare it.

In [ ]:
agents.add_action_group_with_lambda(
    agent_name=mortgage_application_agent_name,
    lambda_function_name=mortgage_application_lambda_name,
    source_code_file="mortgage_application_function.py",
    agent_functions=functions_def,
    agent_action_group_name="mortgage_application_actions",
    agent_action_group_description="Function to manage applications for a user "
)

Step 8: Lets do a quick test

In [ ]:
response = agents.invoke(
    """Hi, I am interested in getting a home loan. Can you help?""", 
    mortgage_application_agent_id, enable_trace=True
)
print("====================")
print(response)

Step 9 : Create alias of the agent.

After you create an alias, you can use the agent in your application by invokaing the agent by its alias. This allows you to update the agent's versions without making changes in your application.

In [ ]:
mortgage_application_agent_alias, mortgage_application_agent_alias_arn = agents.create_agent_alias(
    mortgage_application_agent_id, 'mortgage_application_agent_a'
)

Step 10 : Saving information

Lets store agent information to be used in subsequent labs.

In [ ]:
%store mortgage_application_agent_alias_arn


Step 11 : Testing the agent - Bedrock Console

To test the agent, follow these steps:
1. Open the AWS console on a separate tab (right click on the aws icon on top left and click on 'open in new tab')
2. Ensure that you are in AWS region: United States (oregon) us-west-2 (you can change the region from the drop down menu at top right)
3. Open Amazon Bedrock (you can search for it from the search bar at the top)

4. On the Bedrock console, expand the left navigation panel and click 'Agents' under 'Build' section (see 1 in screenshot below)
5. Click 'mortgage_applications_agent' to open the agent details page (see 2 in screenshot below)

<img src="../../images/nb_1_1_bedrock_console.jpg" width="500"/>

6. Click the Test button (see 3 in screenshot below)
7. You can chat with the agent by typing in the text box in the Test panel (see 4 in screenshot below).
8. Start with the prompt 'I need to apply for a new mortgage'

<img src="../../images/nb_1_2_agent_test.jpg" width="500"/>

The next section expands the single agent to a multi agent collaboration system that can address multiple types of queries.

Start with the Readme.md file in 2_bedrock-multi-agent folder.